In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
#from loguru import logger
from typing import Union, Optional
from fastapi.middleware.cors import CORSMiddleware

from oct2py import octave

octave.run('BattMo/startupBattMo.m')

main_app = FastAPI()
app = FastAPI(title="BattMo API", description="runs BattMo Simulations as a service", version="0.1.0")

origins = [
    "*",
    "https://api.battmo.open-semantic-lab.org",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class TestRequest(BaseModel):
    text: str

class ActiveMaterial(BaseModel):
    thickness: float
    
class Separator(BaseModel):
    thickness: float

class Geometry1D(BaseModel):
    format: Optional[str] = "1D"
    faceArea: Optional[float] = 1e-4
    NegativeElectrode: Optional[ActiveMaterial] = ActiveMaterial(thickness = 64e-6)
    PositiveElectrode: Optional[ActiveMaterial] = ActiveMaterial(thickness = 57e-6)
    Electrolyte: Optional[Separator] = Separator(thickness = 15e-6)
    
class PerformanceSpec(BaseModel):
    E: Optional[float]
    energyDensity: Optional[float]
    energy: Optional[float]
    
class PerformanceSpecRequest(BaseModel):
    geometry: Optional[Geometry1D] = Geometry1D()
    
class PerformanceSpecResponse(BaseModel):
    status: Optional[str] = "ok"
    result: PerformanceSpec

@app.get("/")
def read_root():
    return {"Hello": "Root"}

@app.get("/fastapi/")
def read_fastapi():
    return {"Hello": "World2"}

@app.get("/items/{item_id}")
def read_item(item_id: int, q: Union[str, None] = None):
    return {"item_id": item_id, "q": q}
    
@app.post("/test")
def index(request: TestRequest):
    #logger.debug(request)
    return {"ok": True}

@app.post("/run/performance-spec", response_model=PerformanceSpecResponse)
def performance_spec(request: PerformanceSpecRequest):
    #ToDo
    E, energyDensity, energy = octave.runJsonFunction({'ParameterData/BatteryCellParameters/LithiumIonBatteryCell/lithium_ion_battery_nmc_graphite.json', 'Examples/utils/data/geometry1d.json'}, nout=3)
    response = PerformanceSpecResponse(result=PerformanceSpec(E = E[0][-1], energyDensity = energyDensity[0][-1], energy = energy[0][-1]))
    return response

main_app.mount("/api", app)

import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(main_app, port=8200, host="0.0.0.0")

#for testing open https://api.battmo.open-semantic-lab.org/api/docs#/default/performance_spec_run_performance_spec_post

Welcome to the MATLAB Reservoir Simulation Toolbox (MRST)!
You are using the release version 2021b. To download other versions of MRST
and view examples and relevant publications, please visit www.mrst.no

Useful commands for getting started:
 - List all introductory examples:   mrstExamples()
 - List all modules:                 mrstPath('list')
 - Load modules using GUI:           mrstModule('gui')
 - Explore all available data sets:  mrstDatasetGUI()
 - List examples of a module:        mrstExamples('ad-blackoil')
 - Explore modules and publications: mrstExploreModules()
 - Show all examples in all modules: mrstExamples('all')
 - Show settings for MRST:           mrstSettings()
 - Display this message:             mrstStartupMessage()

For assistance and discussions about MRST, please visit our mailing list at
	www.sintef.no/projectweb/mrst/forum/ (sintef-mrst@googlegroups.com)
For some common queries, see our FAQ: www.sintef.no/projectweb/mrst/faq/


INFO:     Started server process [1923621]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8200 (Press CTRL+C to quit)


Solving timestep 01/50:                      -> 100 Seconds, 799 Milliseconds
Solving timestep 02/50: 100 Seconds, 799 Milliseconds -> 201 Seconds, 599 Milliseconds
Solving timestep 03/50: 201 Seconds, 599 Milliseconds -> 302 Seconds, 399 Milliseconds
Solving timestep 04/50: 302 Seconds, 399 Milliseconds -> 403 Seconds, 199 Milliseconds
Solving timestep 05/50: 403 Seconds, 199 Milliseconds -> 504 Seconds
Solving timestep 06/50: 504 Seconds          -> 604 Seconds, 799 Milliseconds
Solving timestep 07/50: 604 Seconds, 799 Milliseconds -> 705 Seconds, 599 Milliseconds
Solving timestep 08/50: 705 Seconds, 599 Milliseconds -> 806 Seconds, 399 Milliseconds
Solving timestep 09/50: 806 Seconds, 399 Milliseconds -> 907 Seconds, 199 Milliseconds
Solving timestep 10/50: 907 Seconds, 199 Milliseconds -> 1007 Seconds, 999 Milliseconds
Solving timestep 11/50: 1007 Seconds, 999 Milliseconds -> 1108 Seconds, 799 Milliseconds
Solving timestep 12/50: 1108 Seconds, 799 Milliseconds -> 1209 Seconds, 599 